In [15]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np



In [16]:
# 2021 Lottery data file
wa_lottery_to_load = Path("Resources/2021_lottery_data.csv")

#Read the lottery data file and convert into a single DataFrame
wa_lottery_df = pd.read_csv(wa_lottery_to_load)

# Display the data table for preview
wa_lottery_df

,date,weekday,winning_numbers,powerball,powerplay,jackpot
0,1/2/2021,Sat,3-4-11-41-67,5,2,384000000
1,1/6/2021,Wed,1-20-22-60-66,3,3,410000000
2,1/9/2021,Sat,14-26-38-45-46,13,2,470000000
3,1/13/2021,Wed,4-19-23-25-49,14,2,550000000
4,1/16/2021,Sat,14-20-39-65-67,2,3,640000000
...,...,...,...,...,...,...
118,12/20/2021,Mon,2-13-23-34-66,2,4,363000000
119,12/22/2021,Wed,7-16-19-48-68,15,2,378000000
120,12/25/2021,Sat,27-29-45-55-58,2,3,400000000
121,12/27/2021,Mon,36-38-45-62-64,19,2,416000000


In [55]:
# Created bar plot showing the most played powerball/powerplay number for each weekday using Pandas.
# Need to use .groupby plus .count to get the number of powerball plays for each weekday
weekday_powerball_total = wa_lottery_df.groupby(["weekday"])["powerball"].value_counts()

#Take .max to find most played powerball number 
max_powerball = weekday_total.groupby(["weekday"]).max()
max_powerball

weekday
Mon    3
Sat    5
Wed    5
Name: powerball, dtype: int64

In [54]:
# Need to use .groupby plus .count to get the number of powerplay plays for each weekday
weekday_powerplay_total = wa_lottery_df.groupby(["weekday"])["powerplay"].value_counts()

#Take .max to find most played powerplay number 
most_powerplay = bar_chart_grouped.groupby(["weekday"]).max()
max_powerplay

weekday
Mon    12
Sat    30
Wed    28
Name: powerplay, dtype: int64

In [ ]:
# Plot bar chart
data = df.plot(kind = 'barh', ylabel = 'weekday', figsize = (10,5))
data.set(xlabel = 'Most Played Number')
for c in data.containers:
    data.bar_label(c, fmt = '%.0f', label_type = 'edge')
data.margins(x = 0.1)
data.lengend(title = 'Lottery', bbox_to_anchor = (1, 1.02), loc = 'upper left')


plt.show()
plt.tight_layout()